## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hermanus,ZA,-34.4187,19.2345,63.90,74,11,4.03,few clouds
1,Tiksi,RU,71.6872,128.8694,39.27,90,100,6.53,overcast clouds
2,Fort Nelson,CA,58.8053,-122.7002,60.39,63,40,3.44,scattered clouds
3,Bredasdorp,ZA,-34.5322,20.0403,65.52,59,38,5.97,scattered clouds
4,Guaymas,MX,27.9333,-110.9000,78.89,73,1,0.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 77


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,Portland,US,45.5234,-122.6762,71.55,67,20,1.99,few clouds
9,Pangai,TO,-19.8000,-174.3500,72.01,96,100,15.68,very heavy rain
12,Nishihara,JP,26.1842,127.7558,76.28,94,90,14.70,light intensity drizzle
17,Rikitea,PF,-23.1203,-134.9692,71.73,58,61,21.25,broken clouds
21,Kiunga,PG,-6.1219,141.2906,72.68,99,84,2.86,broken clouds
24,Grafton,US,42.2070,-71.6856,71.74,65,100,1.99,overcast clouds
28,Mahebourg,MU,-20.4081,57.7000,70.09,94,75,6.91,moderate rain
30,Saint-Philippe,RE,-21.3585,55.7679,74.79,78,42,10.22,scattered clouds
33,Virginia Beach,US,36.8529,-75.9780,74.05,62,40,4.61,scattered clouds
43,Torbay,CA,47.6666,-52.7314,70.57,82,90,27.63,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,Portland,US,45.5234,-122.6762,71.55,67,20,1.99,few clouds
9,Pangai,TO,-19.8000,-174.3500,72.01,96,100,15.68,very heavy rain
12,Nishihara,JP,26.1842,127.7558,76.28,94,90,14.70,light intensity drizzle
17,Rikitea,PF,-23.1203,-134.9692,71.73,58,61,21.25,broken clouds
21,Kiunga,PG,-6.1219,141.2906,72.68,99,84,2.86,broken clouds
24,Grafton,US,42.2070,-71.6856,71.74,65,100,1.99,overcast clouds
28,Mahebourg,MU,-20.4081,57.7000,70.09,94,75,6.91,moderate rain
30,Saint-Philippe,RE,-21.3585,55.7679,74.79,78,42,10.22,scattered clouds
33,Virginia Beach,US,36.8529,-75.9780,74.05,62,40,4.61,scattered clouds
43,Torbay,CA,47.6666,-52.7314,70.57,82,90,27.63,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Portland,US,71.55,few clouds,45.5234,-122.6762,
9,Pangai,TO,72.01,very heavy rain,-19.8000,-174.3500,
12,Nishihara,JP,76.28,light intensity drizzle,26.1842,127.7558,
17,Rikitea,PF,71.73,broken clouds,-23.1203,-134.9692,
21,Kiunga,PG,72.68,broken clouds,-6.1219,141.2906,
24,Grafton,US,71.74,overcast clouds,42.2070,-71.6856,
28,Mahebourg,MU,70.09,moderate rain,-20.4081,57.7000,
30,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,
33,Virginia Beach,US,74.05,scattered clouds,36.8529,-75.9780,
43,Torbay,CA,70.57,light rain,47.6666,-52.7314,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")    

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_without_missing = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df =hotel_without_missing.dropna(how='any')
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Portland,US,71.55,few clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
9,Pangai,TO,72.01,very heavy rain,-19.8000,-174.3500,Ha'apai Beach Resort
12,Nishihara,JP,76.28,light intensity drizzle,26.1842,127.7558,Minshuku Agaihama
17,Rikitea,PF,71.73,broken clouds,-23.1203,-134.9692,Pension Maro'i
21,Kiunga,PG,72.68,broken clouds,-6.1219,141.2906,Airport Lodge Kiunga
24,Grafton,US,71.74,overcast clouds,42.2070,-71.6856,The Grafton Inn
28,Mahebourg,MU,70.09,moderate rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
30,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
33,Virginia Beach,US,74.05,scattered clouds,36.8529,-75.9780,Hilton Virginia Beach Oceanfront
43,Torbay,CA,70.57,light rain,47.6666,-52.7314,Gallows Cove


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))